<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/4.%20Bike_Sharing_Demand_improve_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [25]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
!kaggle competitions list

In [27]:
!kaggle competitions download -c bike-sharing-demand

bike-sharing-demand.zip: Skipping, found more recently modified local copy (use --force to force download)


In [28]:
!ls

 Bike_Sharing_Demand_submission12.csv   kaggle.json	       test.csv
 bike-sharing-demand.zip	        sample_data	       train.csv
'kaggle (1).json'		        sampleSubmission.csv


In [29]:
!unzip bike-sharing-demand.zip

Archive:  bike-sharing-demand.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


# **1. 두 번째 성능 개선 절차**

두 번째 성능 개선 챕터에서는 'month' 피처를 생성하지 않거나 'season' 피처를 제거할 것입니다. 'month' 피처를 네 구간으로 나누면 'season' 피처인 사계절이 되기 때문입니다. 물론 성능이 향상될지는 알 수 없습니다. 머신러닝에서는 상관성이 높은 피처 중 하나를 제거한다고 무조건 성능이 향상되지 않습니다. 오히려 성능이 저하되는 경우도 많습니다.

# **2. 성능 개선을 위한 피처 엔지니어링 - 'month' 피처 생성 제외**

In [ ]:
import numpy as np
import pandas as pd
import random

np.random.seed(2022)
random.seed(2022)

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 50)
pd.set_option('display.max_rows', 50)

# 데이터를 읽습니다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sampleSubmission.csv')

## **2.1. 이상치 레코드 제거**

'weather' 피처에서 값이 4인 피처를 제거하겠습니다.

In [ ]:
train = train[train['weather']!=4]

## **2.2. 훈련 및 테스트 세트 결합**

피처 엔지니어링을 수행하기 위해 훈련 세트와 테스트 세트를 결합하겠습니다.

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3.0,13.0,16.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8.0,32.0,40.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5.0,27.0,32.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3.0,10.0,13.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17373,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
17374,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
17375,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014,NaN,NaN,NaN
17376,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981,NaN,NaN,NaN


## **2.3. 피처 분할**

'datetime' 피처를 연, 월, 시, 요일 피처로 분할하겠습니다. 사용하기에 부적합한 날짜, 일, 분, 초 피처는 생성하지 않겠습니다.

In [ ]:
from datetime import datetime
import calendar

# 연, 월, 시 피처를 생성합니다.
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])

# 날짜를 추출하고 날짜에 해당하는 요일을 숫자로 치환합니다.
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
all_data['day_of_week'] = all_data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())

# 날짜 피처를 삭제합니다.
all_data.drop(['datetime', 'date'], axis=1, inplace=True)

In [ ]:
all_data.head(1)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,hour,day_of_week
0,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0,2011,00,5


## **2.4. 불필요한 피처 제거**

불필요한 피처를 제거하겠습니다.

In [ ]:
feature_list = ['casual', 'windspeed', 'registered']
all_data.drop(feature_list, axis=1, inplace=True)

## **2.5. 피처 인코딩**

파생 피처인 'year', 'hour'는 명목형 피처입니다. 원-핫 인코딩을 적용하겠습니다.

In [ ]:
all_data_ohe = pd.get_dummies(all_data)
all_data_ohe.head()

,season,holiday,workingday,weather,temp,atemp,humidity,count,day_of_week,year_2011,year_2012,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,1,0,0,1,9.84,14.395,81,16.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,9.02,13.635,80,40.0,5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,9.02,13.635,80,32.0,5,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,9.84,14.395,75,13.0,5,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,9.84,14.395,75,1.0,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **3. 모델링**

## **3.1. 데이터 준비**

데이터를 준비하겠습니다.

In [ ]:
train_num = len(train) # 훈련 세트 개수를 지정합니다.
X_train_df = all_data_ohe[:train_num].drop('count', axis=1) # 훈련 세트를 지정합니다.
X_test_df = all_data_ohe[train_num:].drop('count', axis=1) # 테스트 세트를 지정합니다.
y_train = train['count'] # 타깃값을 지정합니다.

타깃값에 로그 변환을 적용하겠습니다.

In [ ]:
log_y_train = np.log(y_train)

## **3.2. XGBoost**

### **3.2.1. 하이퍼파라미터 튜닝**

optuna를 사용하여 하이퍼파라미터를 튜닝하겠습니다.

In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 12.6 MB/s 
     |████████████████████████████████| 81 kB 10.1 MB/s 
     |████████████████████████████████| 210 kB 75.2 MB/s 
     |████████████████████████████████| 78 kB 5.2 MB/s 
     |████████████████████████████████| 147 kB 72.7 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 112 kB 67.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=718a342abd27fe19213a8164c18b37b4f7be73f1a8eb7a2438bc9c2b578d3714
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0


이 문제의 측정 지표는 RMSLE를 계산하는 함수를 선언하겠습니다.

In [ ]:
def rmsle(y, prediction, exponent=True): # 지수 변환을 기본값으로 지정합니다.
    # 타깃값에 지수 변환을 수행하길 원하면 지수 변환을 수행합니다.
    if exponent:
        y = np.exp(y)
        prediction = np.exp(prediction)

    # RMSLE 공식을 구현합니다.
    # 로그 변환을 수행하고 넘파이의 nan_to_num() 메서드를 사용하여 결측치를 0으로 변환합니다.
    log_y = np.nan_to_num(np.log(y + 1))
    log_prediction = np.nan_to_num(np.log(prediction + 1))
    result = np.sqrt(np.mean((log_y - log_prediction)**2))

    return result

하이퍼파라미터 튜닝을 수행하겠습니다.

In [ ]:
import optuna
from optuna.samplers import TPESampler

# 시도 과정을 출력하지 않는 코드입니다.
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

def objective_XGB(trial, X_train_df, log_y_train):
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, log_y_train,
                                                          test_size=0.2,
                                                          random_state=42)
    
    params = {
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 100),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 100),
        'gamma': trial.suggest_float('gamma', 0, 9),
        'random_state': 42
    }
    model = XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
              early_stopping_rounds=50,
              verbose=False)
    prediction = model.predict(X_valid)
    rmsle_score = rmsle(y_valid, prediction)

    return rmsle_score

In [ ]:
study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(lambda trial: objective_XGB(trial, X_train_df, log_y_train),
               n_trials=50,
               show_progress_bar=True)

print(f'Best trial score: {study.best_trial.value}')
print(f'Best params: {study.best_trial.params}')

/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

Best trial score: 0.352631119030974
Best params: {'booster': 'gbtree', 'learning_rate': 0.05128408723604819, 'n_estimators': 1736, 'max_depth': 9, 'subsample': 0.8685343290933354, 'colsample_bytree': 0.7165991039694154, 'reg_alpha': 0.3782504135120037, 'reg_lambda': 0.9772975944749582, 'gamma': 4.651734341297651}


파라미터 중요도를 시각화해 보겠습니다.

In [ ]:
optuna.visualization.plot_param_importances(study)

### **3.2.2. 모델 훈련**

최적 파라미터를 적용하여 모델을 생성하겠습니다.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, log_y_train,
                                                      test_size=0.2,
                                                      random_state=42)

params = study.best_trial.params
xgb_reg_model = XGBRegressor(**params)
xgb_reg_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  early_stopping_rounds=50, verbose=False)

### **3.2.3. 모델 점수 확인**

제출 파일을 생성하고 점수를 확인하겠습니다.

In [ ]:
xgb_prediction_test = xgb_reg_model.predict(X_test_df)
submission['count'] = np.exp(xgb_prediction_test)
submission.to_csv('Bike_Sharing_Demand_submission11.csv', index=False)

프라이빗 스코어는 0.39895입니다. 'month' 피처를 생성했을 경우인 0.39687보다 오히려 약간 상승했습니다.

## **3.3. LightGBM**

### **3.3.1. 하이퍼파라미터 튜닝**

In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

def objective_LGBM(trial, X_train_df, log_y_train):
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, log_y_train,
                                                          test_size=0.2,
                                                          random_state=42)
        
    params = {
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 10),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'max_depth': -1,
        'subsample': trial.suggest_float('subsample', 0.6, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 100),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 50, 1000),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10),
        'random_state': 42
    }
    model = LGBMRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
              early_stopping_rounds=50,
              verbose=False)
    prediction = model.predict(X_valid)
    rmsle_score = rmsle(y_valid, prediction)

    return rmsle_score

In [ ]:
study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(lambda trial: objective_LGBM(trial, X_train_df, log_y_train),
               n_trials=500,
               show_progress_bar=True)

print(f'Best trial score: {study.best_trial.value}')
print(f'Best params: {study.best_trial.params}')

/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')
<ipython-input-17-e7eb54d78fe1>:5: RuntimeWarning: overflow encountered in exp
  prediction = np.exp(prediction)
<ipython-input-17-e7eb54d78fe1>:11: RuntimeWarning: overflow encountered in square
  result = np.sqrt(np.mean((log_y - log_prediction)**2))


Best trial score: 0.3011108375920747
Best params: {'boosting_type': 'gbdt', 'learning_rate': 0.21138101737594708, 'n_estimators': 778, 'num_leaves': 107, 'subsample': 0.701330592988819, 'colsample_bytree': 0.8106435329332532, 'reg_alpha': 0.11557306525638887, 'reg_lambda': 62.15376754866887, 'min_child_samples': 170, 'min_child_weight': 3.8682542969100218}


In [ ]:
optuna.visualization.plot_param_importances(study)

[W 2023-01-03 14:15:10,669] Trial 1 is omitted in visualization because its objective value is inf or nan.
[W 2023-01-03 14:15:10,670] Trial 3 is omitted in visualization because its objective value is inf or nan.
[W 2023-01-03 14:15:10,671] Trial 8 is omitted in visualization because its objective value is inf or nan.
[W 2023-01-03 14:15:10,677] Trial 9 is omitted in visualization because its objective value is inf or nan.
[W 2023-01-03 14:15:10,678] Trial 37 is omitted in visualization because its objective value is inf or nan.


### **3.3.2. 모델 훈련**

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, log_y_train,
                                                      test_size=0.2,
                                                      random_state=42)

params = study.best_trial.params
lgbm_reg_model = LGBMRegressor(**params)
lgbm_reg_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  early_stopping_rounds=50, verbose=False)

LGBMRegressor(colsample_bytree=0.8106435329332532,
              learning_rate=0.21138101737594708, min_child_samples=170,
              min_child_weight=3.8682542969100218, n_estimators=778,
              num_leaves=107, reg_alpha=0.11557306525638887,
              reg_lambda=62.15376754866887, subsample=0.701330592988819)

### **3.3.3. 모델 점수 확인**

In [ ]:
lgbm_prediction_test = lgbm_reg_model.predict(X_test_df)
submission['count'] = np.exp(lgbm_prediction_test)
submission.to_csv('Bike_Sharing_Demand_submission12.csv', index=False)

프라이빗 스코어는 0.39867입니다. 'month' 피처를 생성했을 경우인 0.40596보다 하락했습니다.

# **4. 성능 개선을 위한 피처 엔지니어링 - 'season' 피처 제거**

In [30]:
import numpy as np
import pandas as pd
import random

np.random.seed(2022)
random.seed(2022)

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 50)
pd.set_option('display.max_rows', 50)

# 데이터를 읽습니다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sampleSubmission.csv')

## **4.1. 이상치 레코드 제거**

'weather' 피처에서 값이 4인 피처를 제거하겠습니다.

In [31]:
train = train[train['weather']!=4]

## **4.2. 훈련 및 테스트 세트 결합**

피처 엔지니어링을 수행하기 위해 훈련 세트와 테스트 세트를 결합하겠습니다.

In [32]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3.0,13.0,16.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8.0,32.0,40.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5.0,27.0,32.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3.0,10.0,13.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17373,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
17374,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
17375,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014,NaN,NaN,NaN
17376,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981,NaN,NaN,NaN


## **4.3. 피처 분할**

'datetime' 피처를 연, 월, 시, 요일 피처로 분할하겠습니다. 사용하기에 부적합한 날짜, 일, 분, 초 피처는 생성하지 않겠습니다.

In [33]:
from datetime import datetime
import calendar

# 연, 월, 시 피처를 생성합니다.
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])

# 날짜를 추출하고 날짜에 해당하는 요일을 숫자로 치환합니다.
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
all_data['day_of_week'] = all_data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())

# 날짜 피처를 삭제합니다.
all_data.drop(['datetime', 'date'], axis=1, inplace=True)

In [34]:
all_data.head(1)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,day_of_week
0,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0,2011,01,00,5


## **4.4. 불필요한 피처 제거**

불필요한 피처를 제거하겠습니다.

In [35]:
feature_list = ['season', 'casual', 'windspeed', 'registered']
all_data.drop(feature_list, axis=1, inplace=True)

## **4.5. 피처 인코딩**

파생 피처인 'year', 'hour'는 명목형 피처입니다. 원-핫 인코딩을 적용하겠습니다.

In [36]:
all_data_ohe = pd.get_dummies(all_data)
all_data_ohe.head()

,holiday,workingday,weather,temp,atemp,humidity,count,day_of_week,year_2011,year_2012,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0,0,1,9.84,14.395,81,16.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,9.02,13.635,80,40.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,9.02,13.635,80,32.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,9.84,14.395,75,13.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,9.84,14.395,75,1.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
lgbm_prediction_test = lgbm_reg_model.predict(X_test_df)
submission['count'] = np.exp(lgbm_prediction_test)
submission.to_csv('Bike_Sharing_Demand_submission12.csv', index=False)

프라이빗 스코어는 0.40596입니다. 파라미터 튜닝을 수행하기 전인 0.40073보다 오히려 약간 증가했습니다.

# **5. 모델링**

## **5.1. 데이터 준비**

데이터를 준비하겠습니다.

In [38]:
train_num = len(train) # 훈련 세트 개수를 지정합니다.
X_train_df = all_data_ohe[:train_num].drop('count', axis=1) # 훈련 세트를 지정합니다.
X_test_df = all_data_ohe[train_num:].drop('count', axis=1) # 테스트 세트를 지정합니다.
y_train = train['count'] # 타깃값을 지정합니다.

타깃값에 로그 변환을 적용하겠습니다.

In [39]:
log_y_train = np.log(y_train)

## **5.2. XGBoost**

### **5.2.1. 하이퍼파라미터 튜닝**

optuna를 사용하여 하이퍼파라미터를 튜닝하겠습니다.

In [40]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


이 문제의 측정 지표는 RMSLE를 계산하는 함수를 선언하겠습니다.

In [41]:
def rmsle(y, prediction, exponent=True): # 지수 변환을 기본값으로 지정합니다.
    # 타깃값에 지수 변환을 수행하길 원하면 지수 변환을 수행합니다.
    if exponent:
        y = np.exp(y)
        prediction = np.exp(prediction)

    # RMSLE 공식을 구현합니다.
    # 로그 변환을 수행하고 넘파이의 nan_to_num() 메서드를 사용하여 결측치를 0으로 변환합니다.
    log_y = np.nan_to_num(np.log(y + 1))
    log_prediction = np.nan_to_num(np.log(prediction + 1))
    result = np.sqrt(np.mean((log_y - log_prediction)**2))

    return result

하이퍼파라미터 튜닝을 수행하겠습니다.

In [42]:
import optuna
from optuna.samplers import TPESampler

# 시도 과정을 출력하지 않는 코드입니다.
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [43]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

def objective_XGB(trial, X_train_df, log_y_train):
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, log_y_train,
                                                          test_size=0.2,
                                                          random_state=42)
    
    params = {
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 100),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 100),
        'gamma': trial.suggest_float('gamma', 0, 9),
        'random_state': 42
    }
    model = XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
              early_stopping_rounds=50,
              verbose=False)
    prediction = model.predict(X_valid)
    rmsle_score = rmsle(y_valid, prediction)

    return rmsle_score

In [45]:
study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(lambda trial: objective_XGB(trial, X_train_df, log_y_train),
               n_trials=50,
               show_progress_bar=True)

print(f'Best trial score: {study.best_trial.value}')
print(f'Best params: {study.best_trial.params}')

  0%|          | 0/50 [00:00<?, ?it/s]

Best trial score: 0.31765993733068637
Best params: {'booster': 'gbtree', 'learning_rate': 0.05476934061852211, 'n_estimators': 664, 'max_depth': 5, 'subsample': 0.676862465998019, 'colsample_bytree': 0.7102492575514119, 'reg_alpha': 0.9251015380018277, 'reg_lambda': 12.758841940304231, 'gamma': 0.5709619438782282}


파라미터 중요도를 시각화해 보겠습니다.

In [46]:
optuna.visualization.plot_param_importances(study)

### **5.2.2. 모델 훈련**

최적 파라미터를 적용하여 모델을 생성하겠습니다.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, log_y_train,
                                                      test_size=0.2,
                                                      random_state=42)

params = study.best_trial.params
xgb_reg_model = XGBRegressor(**params)
xgb_reg_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  early_stopping_rounds=50)

### **5.2.3. 모델 점수 확인**

제출 파일을 생성하고 점수를 확인하겠습니다.

In [48]:
xgb_prediction_test = xgb_reg_model.predict(X_test_df)
submission['count'] = np.exp(xgb_prediction_test)
submission.to_csv('Bike_Sharing_Demand_submission13.csv', index=False)

프라이빗 스코어는 0.44442입니다. 'month' 피처를 생성하지 않은 0.39895보다 큰 폭으로 감소했습니다. 'season' 피처의 영향력이 매우 큰 것으로 판단되므로 LightGBM 모델까지 실험해 볼 필요가 없습니다.